## 6.4. 循环神经网络的从零开始实现

In [1]:
import d2lzh as d2l
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as K
import time

(corpus_indices, corpus_chars, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

num_steps = 3 # 通过多少个字预测下一个字

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### 6.4.1 one-hot向量

In [2]:
keras.utils.to_categorical(np.array([0, 2]), vocab_size)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

### 6.4.2 初始化模型参数
### 6.4.3 定义模型

In [3]:
class RNNModel(keras.Model):
    def __init__(self, vocab_size, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rnn = keras.layers.SimpleRNN(256, input_shape=(num_steps, vocab_size))
        self.vocab_size = vocab_size
        self.dense = keras.layers.Dense(vocab_size)
        self.softmax = keras.layers.Softmax()
        
    def call(self, inputs, training=None, mask=None):
        x = self.rnn(inputs)
        x = self.dense(x)
        x = self.softmax(x)
        return x
        
model = RNNModel(vocab_size)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, idx_to_char, char_to_idx):
    output = np.array([char_to_idx[prefix[idx]] for idx in range(len(prefix))])
    for t in range(num_chars + len(prefix) - 1):
        # print(output)
        X = keras.utils.to_categorical(output[-num_steps : ], vocab_size)
        # print('X', X.shape, output[-num_steps : ])
        Y = model.predict(X.reshape(1, num_steps, vocab_size))  # 引入batch=1维度
        if t < len(prefix) - 1:
            # output = np.append(output, char_to_idx[prefix[t + 1]])
            pass
        else:
            output = np.append(output, int(Y.argmax(axis=-1)))
    return ''.join([idx_to_char[i] for i in output])

In [5]:
predict_rnn_gluon('分开 ', 10, model, vocab_size, idx_to_char, char_to_idx)

'分开 强限毫伤春谷驳魂B悔'

In [6]:
def train_and_predict_rnn_gluon(model, vocab_size, corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    model.compile(
        optimizer='adam', #keras.optimizers.SGD(learning_rate=lr, momentum=0, decay=0, clipvalue=1), 
        loss=keras.losses.categorical_crossentropy)
    
    for epoch in range(num_epochs):
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps)
        for X, Y in data_iter:
            x = keras.utils.to_categorical(X, vocab_size)
            y = keras.utils.to_categorical(Y[:,-1], vocab_size)
            #print(x.shape, y.shape)
            model.train_on_batch(x.reshape(batch_size, num_steps, vocab_size), y)
        
        # print(epoch, pred_period)
        if (epoch + 1) % pred_period == 0:
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(
                    prefix, pred_len, model, vocab_size, idx_to_char,
                    char_to_idx))
        

In [7]:
num_epochs, batch_size, lr, clipping_theta = 100, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 10, 50, ['分开 ', '不分开'] # num_step固定为3个字符
train_and_predict_rnn_gluon(model, vocab_size, corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)


        

 - 分开 它动我 到不我都不的 爱放在 年 我一能踢力 快人我篮 这传我 成不都 不到 爱却在的迷 作怎我呆 
 - 不分开 它动我 到不我都不的 爱放在 年 我一能踢力 快人我篮 这传我 成不都 不到 爱却在的迷 作怎我呆
 - 分开 它可灌木句旁邂办 一 正气 他经儿子 习什么我想睡你 像我法和寞排棒  想穿你你当很 的吧上 我间耍
 - 不分开 它可灌木句旁邂办 一 正气 他经儿子 习什么我想睡你 像我法和寞排棒  想穿你你当很 的吧上 我间
 - 分开 话可完你人义让 彻道 分三怎话头对你脸依场落  爽穿一驳实气底点经手射去阵堂将吹属 就年开不 口不她
 - 不分开 话可完你人义让 彻道 分三怎话头对你脸依场落  爽穿一驳实气底点经手射去阵堂将吹属 就年开不 口不
 - 分开 了可去她人将江么 我打我妈妈 难说你笔 我真耍的瞎模   古穿主只层太野试是远力透 奔什么爱我不开不
 - 不分开 了可去她人将江么 我打我妈妈 难说你笔 我真耍的瞎模   古穿主只层太野试是远力透 奔什么爱我不开
 - 分开 了可去简以爱句 我办定到 熟却已想别是  透知的子 在什动 我到店分漂 它伤拥的娘只 娘法依娘止 娘
 - 不分开 了可去简以爱句 我办定到 熟却已想别是  透知的子 在什动 我到店分漂 它伤拥的娘只 娘法依娘止 
 - 分开 它可灌木句旁邂办 一 正气 他经笑子我有惯烦恼小 没怎你我有你烦 我有多烦恼多 熬想 穿不时层不停试
 - 不分开不了口让她知道 我的天画在的太受  边穿过默层地壶试鸠 力常在眼多 着想的手不放开 爱可不让以碰神还
 - 分开 它可灌实句旁点办逗射 物堂 它属拥我年只能在怕脑 蜥有横著我 家和的爹棒早已想猜开不透口想子默不什过
 - 不分开 它可灌实句旁点办逗射 物堂 它属拥我年只能在怕脑 蜥有横著我 家和的爹棒早已想猜开不透口想子默不什
 - 分开 了可去简以将句还 说离怎么无有打我妈妈 难说你笔 我真的分用 话说完说过义 慢彻温忆的让有疯狂的可爱
 - 不分开 了可去简以将句还 说离怎么无有打我妈妈 难说你笔 我真的分用 话说完说过义 慢彻温忆的让有疯狂的可
 - 分开 了可去简以爱句想我办不到 什却已想别人 爸么回那看着  想 有不  不知不觉 你已经很是 说了怎么说
 - 不分开 了可去简以爱句想我办不到 什却已想别人 爸么回那